In [7]:
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
import torch


# 使用本地 LLaMA 模型
llm_llama3_8B = ChatOllama(model="llama3:8B")
llm_phi3_3dot8B = ChatOllama(model="phi3:3.8B")
llm_mistral_7B = ChatOllama(model="mistral:7B")
llm_gemma3_4B = ChatOllama(model="gemma3:4B")

# GPU 加速
print("=== PyTorch GPU 加速環境檢查 ===")
print(f"PyTorch 版本: {torch.__version__}")
print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
print(f"是否支援 CUDA: {torch.cuda.is_available()}")

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        current_gpu = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_gpu)

        print(f"偵測到 {gpu_count} 個 GPU")
        print(f"當前使用的 GPU：{device_name}")
        device = torch.device("cuda")
    else:
        raise RuntimeError("CUDA 不可用，將使用 CPU")
except Exception as e:
    print(f"無法使用 GPU：{e}")
    device = torch.device("cpu")

print(f"pytorch可用的GPU為：{device}")


=== PyTorch GPU 加速環境檢查 ===
PyTorch 版本: 2.8.0+cpu
PyTorch 編譯的 CUDA 版本: None
是否支援 CUDA: False
無法使用 GPU：CUDA 不可用，將使用 CPU
pytorch可用的GPU為：cpu


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 建立工具 ###

In [9]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from memory import Memory, MemoryStream
from reflection import ReflectionEngine

# 所有 Agent 共用同一個 MemoryStream，彼此回覆會被寫回 FAISS，後續查詢時能互相參考。
# === 1. 建立共享記憶流 ===
memory_stream = MemoryStream()

# === 2. 封裝記憶檢索工具 ===
memory_tool = Tool(
    name="MemoryRetriever",
    func=lambda q: "\n".join(m.content for m in memory_stream.search_by_vector(q)),
    description="檢索過往的重要對話與事件"
)

# initialize_agent 的 tools 參數必須是 List[Tool]
tools = [memory_tool]


### 建立 Agent ###

In [ ]:
# === 3. 定義多個角色與對應 LLM ===
# 新增角色，加入新的 (角色描述, llm) 配對
roles = {
    "llama":  ("家醫科醫生", llm_llama3_8B),
    "mistral":("內科醫生", llm_mistral_7B),
    "gemma":  ("外科醫生", llm_gemma3_4B)
}

agents = {}
for name, (role_desc, llm_model) in roles.items():
    prompt_prefix = f"你是一位{role_desc}，回答時請保持此角色的語氣與思考方式。"
    
    agents[name] = initialize_agent(
        tools=tools,
        llm=llm_model,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        agent_kwargs={"prefix": prompt_prefix}
    )


In [13]:
# === 4. 對話流程示例 ===
def multi_agent_round(user_input: str):
    print(f"\n=== 使用者輸入 ===\n{user_input}\n")
    # 對每個 agent 輪詢
    for name, agent in agents.items():
        response = agent.run({
            "input": user_input,
            "chat_history": []
        })
        print(f"[{name}] {response}\n")
        # 將回覆寫入共享記憶
        memory_stream.add_memory(
            f"{name} 回覆: {response}",
            importance=5
        )


In [14]:
# === 5. 測試對話 ===
# 如果是 .py 就要加判斷式
# if __name__ == "__main__":
#     multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
#     multi_agent_round("根據剛剛的討論，再給我一段結論。")
multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
multi_agent_round("根據剛剛的討論，再給我一段結論。")


=== 使用者輸入 ===
請大家一起討論氣候變遷對農業的影響。



> Entering new AgentExecutor chain...
```json
{
  "action": "MemoryRetriever",
  "action_input": "Climate Change Impacts on Agriculture"
}
```

This response asks the user to use the MemoryRetriever tool to retrieve information about climate change impacts on agriculture.
Observation: 
Thought:```json
{
  "action": "Final Answer",
  "action_input": "Climate change is expected to have significant impacts on agriculture, including changes in temperature and precipitation patterns, increased frequency and severity of extreme weather events, and shifts in growing seasons and crop yields."
}

> Finished chain.
[llama] Climate change is expected to have significant impacts on agriculture, including changes in temperature and precipitation patterns, increased frequency and severity of extreme weather events, and shifts in growing seasons and crop yields.



> Entering new AgentExecutor chain...
 ```json
{
    "action": "Final Answer",
    "action_input": "

In [15]:
# === 6. 反思與長期記憶整理 ===
reflector = ReflectionEngine(llm=llm_llama3_8B,
                             memory_stream=memory_stream,
                             reflection_threshold=20)
reflector.reflect()

已完成反思，新增記憶：
 - 以下是自然中文歸納的 3 個高層次洞察：
 - 溫室氣候變化對農業的影響將導致耕作模式和收穫量的改变，可能引發食物安全問題。
 - 溫暖的天氣 condiciones 可能創造出某些害蟲和疾病的優越環境，嚴重影響農產品的生產。
 - 為了應對這些負面影響，農民需要采取新型耕作策略，例如培養干旱抗性的作物、實施灌溉系統或採用可持續農業方法，以確保未來食物安全。


['以下是自然中文歸納的 3 個高層次洞察：',
 '溫室氣候變化對農業的影響將導致耕作模式和收穫量的改变，可能引發食物安全問題。',
 '溫暖的天氣 condiciones 可能創造出某些害蟲和疾病的優越環境，嚴重影響農產品的生產。',
 '為了應對這些負面影響，農民需要采取新型耕作策略，例如培養干旱抗性的作物、實施灌溉系統或採用可持續農業方法，以確保未來食物安全。']